# Model testing

Now that all the metadata has been studied some testing with models from diferent sources and original code will be undergone in order to define some models to send to the boada environment.

We will start slow with simple pre-trained models extracted directly from the pytorch environment and build from those up with the modifications that we want to study in the experimentation. Mainly the models tested will be ResNet (in some of it's variants) and some DensNet variants at first with the option of further models if everything goes to plan.

Another objective would be to try to test the difference between transfer learning and full training from scratch, considering some other more sophisticated learning methods like one-shot if there is time.

Finally some degree of localization will be accquired with the advancements of the paper from Selvaraju et al. of Grad-CAM.  Code will be recicled with the original paper code with the necessary modifications in order to apply it to the selected models of the project.


## Preparing the split between test, train and validation

In [ ]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

def undersample_negatives(train_data, ratio=0.3):
    
    negative = train_data[train_data['No Finding'] == 1]
    positive = train_data[train_data['No Finding'] == 0]

    print(f'negative {len(negative)} positive {len(positive)}')
    
    negative = resample(negative, 
                        replace=False,
                        n_samples=int(len(positive)*(ratio / (1- ratio))), # reduce as stated in ratio
                        random_state=42)
    
    print(f'negative {len(negative)} positive {len(positive)}')
    
    return pd.concat([positive, negative])

ratio = 0.3

# Get the labels and read the original metadata
labels = ['No Finding',
          'Atelectasis',
          'Cardiomegaly',
          'Consolidation',
          'Edema',
          'Effusion',
          'Emphysema',
          'Fibrosis',
          'Hernia',
          'Infiltration',
          'Mass',
          'Nodule',
          'Pleural_Thickening',
          'Pneumonia',
          'Pneumothorax']

metadata = pd.read_csv('./labels/Data_Entry_2017_v2020.csv', delimiter=',')

# Encode the labels with multi-label friendly encoding
for label in labels:
    metadata[label] = metadata['Finding Labels'].apply(lambda x: 1 if label in x else 0)

metadata = metadata.drop(columns=['Finding Labels', 'Follow-up #','Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]'])

# Get the test train and val splits according to the patient ID so no patients end up split between groups
gss_test = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_val_idx, test_idx = next(gss_test.split(metadata, groups=metadata['Patient ID']))

train_val_metadata = metadata.iloc[train_val_idx]
test_metadata = metadata.iloc[test_idx]

gss_train_val = GroupShuffleSplit(test_size=0.125, n_splits=1, random_state=42)
train_idx, val_idx = next(gss_train_val.split(train_val_metadata, groups=train_val_metadata['Patient ID']))

train_metadata = train_val_metadata.iloc[train_idx]
val_metadata = train_val_metadata.iloc[val_idx]


# Drop the column of patient ID
train_metadata = train_metadata.drop(columns=['Patient ID'])
val_metadata = val_metadata.drop(columns=['Patient ID'])
test_metadata = test_metadata.drop(columns=['Patient ID'])


# Undersample "No Findings"
train_metadata = undersample_negatives(train_metadata,ratio)
val_metadata = undersample_negatives(val_metadata,ratio)
test_metadata = undersample_negatives(test_metadata,ratio)


#Write all the new metadata as csv to load easier
train_metadata.to_csv('./labels/train_metadata.csv', index=False)
val_metadata.to_csv('./labels/val_metadata.csv', index=False)
test_metadata.to_csv('./labels/test_metadata.csv', index=False)

In [ ]:
print(f'in terms of all train: {len(train_metadata)}, val: {len(val_metadata)}, test: {len(test_metadata)} and total: {len(train_metadata) + len(test_metadata) + len(val_metadata) }')


## Dataset definition and load (dataset.py)

We define the dataset and load from the data entries

In [6]:
import torch
from torch.utils.data import Dataset
import os
import numpy as np
import pandas as pd
from PIL import Image

class ChestXRay(Dataset):
    def __init__(self, df_dir, image_dir, transform=None):
        """
        Args:
            df_dir: Path to the csv file with image names and labels.
            image_dir: Directory with all the images with the labels.
            transform: Optional transform to be applied on a sample.
        """
        self.classes = ['No Finding',
                        'Atelectasis',
                        'Cardiomegaly',
                        'Consolidation',
                        'Edema',
                        'Effusion',
                        'Emphysema',
                        'Fibrosis',
                        'Hernia',
                        'Infiltration',
                        'Mass',
                        'Nodule',
                        'Pleural_Thickening',
                        'Pneumonia',
                        'Pneumothorax']

        
        self.data_frame = pd.read_csv(df_dir)
        self.image_dir = image_dir
        self.transform = transform

        self.size = len(self.data_frame)
        self.labels = np.array(self.data_frame.iloc[:, 1:])
        self.images = self.data_frame.iloc[:,0]
        self.class_count = self.labels.sum(0)
        self.total_labels = self.class_count.sum()
        
        
        

    def __getitem__(self, idx):
        # Get image path
        img_name = os.path.join(self.image_dir, self.images.iloc[idx])
        image = Image.open(img_name).convert("RGB")
        
        # Get labels
        labels = np.array(self.labels[idx])
        
        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        image = np.array(image)
        
        return {'image': image, 'labels': labels}
    
    def __len__(self):
        return self.size

## Various functions (utils.py)

### Data Loaders

In [7]:
import torchvision
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import os

normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])


def make_data_loaders(train_csv,val_csv,image_dir,batch_size,image_size):
    

    train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    normalize
    ])

    val_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    normalize
    ])

    train_dataset = ChestXRay(df_dir=train_csv, image_dir=image_dir, transform=train_transforms)
    val_dataset = ChestXRay(df_dir=val_csv, image_dir=image_dir, transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return {'train': train_loader, 'val': val_loader}, {'train':len(train_dataset),'val':len(val_dataset)}, train_dataset.class_count

### Model Load and Store

In [ ]:
train_csv = './labels/train_metadata.csv'
val_csv = './labels/val_metadata.csv'
image_dir = './resized_images'

dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv,val_csv,image_dir,128,256)

print(class_counts)
print(class_counts.shape)

In [9]:
import torch
import os

def LoadModel(checkpoint_file, model, optimizer, epoch_inti, num_GPU):
    '''
    The loads the model, optimizer, current epoch, and current validation AUC from the checkpoint location provided.

    Parameters
    ----------
    checkpoint_file: (str) the location of the model in s/m
    model: PyTorch model
    optimizer: PyTorch optimizer
    epoch_inti: current epoch
    num_GPU : (int) number of GPUs that are being used

    Returns
    -------
    Returns the model, optimizer, epoch_inti, best_auc_ave from the saved location.
    '''
    checkpoint = torch.load(checkpoint_file)
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_inti = checkpoint['epoch']
    if num_GPU > 1:
        model.module.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint['model_state_dict'])
    return model, optimizer, epoch_inti


def SaveModel(model, optimizer, epoch, file_name, num_GPU):
    """
    Save the model parameters, optimizer, best_AUC

    Parameters
    ----------
    epoch : (int) current epoch
    model : Pytorch model to save
    optimizer : Pytorch optimzer to save
    file_name : (str) location where the model needed to be saved
    num_GPU : (int) number of GPUs that are being used

    """
    if num_GPU > 1:
        state = {
            'epoch': epoch,
            'model_state_dict': model.module.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    else:
        state = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    torch.save(state, file_name)
    pass

### Get Optimizer

In [10]:
from torch.optim import SGD, Adadelta, Adam, RMSprop, AdamW

def get_optimizer(params, optimizer, lr=1e-4, momentum=0.9, weight_decay=0.0):
    """
    Loads and returns the optimizer.
    """
    if optimizer == 'SGD':
        return SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer == 'SGD_Nesterov':
        return SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=True)
    elif optimizer == 'Adadelta':
        return Adadelta(params, lr=lr, weight_decay=weight_decay)
    elif optimizer == 'Adam':
        return Adam(params, lr=lr, weight_decay=weight_decay)
    elif optimizer == 'AdamW':
        return AdamW(params, lr=lr, weight_decay=weight_decay)
    elif optimizer == 'RMSprop':
        return RMSprop(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise Exception('Unknown optimizer : {}'.format(optimizer))

### Get Loss

In [11]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd


# Code extracted from MIT licensed code Copyright (c) 2021 Alinstein Jose
# Implementation of "Weakly supervised Classification and Localization of Chest X-ray images"
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations
    predicted : Matrix with predicted data, where rows are observations
    Returns
    -------
    list type, with optimal cutoff value
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr))
    roc = pd.DataFrame({'tf': pd.Series(tpr - (1 - fpr), index=i), 'threshold': pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf - 0).abs().argsort()[:1]]
    return list(roc_t['threshold'])


# Idea extacted from the [https://arxiv.org/pdf/1901.05555] paper "Class-Balanced Loss Based on Effective Number of Samples"
# We can normalize inverse frequencies but this method is studied to do better in other works
def effective_weights(class_counts, beta=0.999):
    effective_num = 1.0 - np.power(beta, class_counts)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights)  # Normalize the weights -> or other(?)
    return torch.tensor(weights, dtype=torch.float32)  # Convert to tensor


# Code extracted from MIT licensed code Copyright (c) 2020 Alibaba-MIIL
# Original idea in the [https://arxiv.org/pdf/2009.14119] paper "Asymmetric Loss For Multi-Label Classification"
class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=True, average=False):
        super(AsymmetricLoss, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps
        self.average = average

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        # Calculating Probabilities
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic CE calculation
        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)  # pt = p if t > 0 else 1-p
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            loss *= one_sided_w

        # Really small change introduced by thesis author (me) to try average loss impact
        # As discussed in "https://github.com/Alibaba-MIIL/ASL/issues/22" it's not suposed
        # To have an impact but I want to be able to use mean for ease of comparison.
        if self.average:
            return -loss.mean()
        else:
            return -loss.sum()
    

# Function for testing different loss types
def get_loss(loss_type='bce', counts=None, beta=0.99, device='cpu'):
    
    if loss_type == 'bce':
        loss_fn = nn.BCEWithLogitsLoss()
    elif loss_type == 'bce_w':
        if counts is not None:
            class_weights = effective_weights(counts, beta=beta).to(device)
            loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)
        else:
            print("You didn't provide weights so we proceed with normal BCE")
            loss_fn = nn.BCEWithLogitsLoss()
    elif loss_type == 'asymmetric':
        loss_fn = AsymmetricLoss()
    elif loss_type == 'asymmetric_avg':
        loss_fn = AsymmetricLoss(average=True)
    elif loss_type == 'asl1':
        loss_fn = AsymmetricLoss(gamma_neg=2, gamma_pos=1, clip=0)
    elif loss_type == 'asl2':
        loss_fn = AsymmetricLoss(gamma_neg=4, gamma_pos=1, clip=0.05)
    elif loss_type == 'asl3':
        loss_fn = AsymmetricLoss(gamma_neg=8, gamma_pos=1, clip=0.05)
    elif loss_type == 'focal':
        loss_fn = AsymmetricLoss(gamma_neg=2, gamma_pos=2, clip=0)
    
    else:
        raise ValueError(f"Unknown loss_type '{loss_type}'. Use 'bce' or 'asymmetric'.")
    
    return loss_fn

In [7]:
import torch
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, steps=1000, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())  # Keep track of best model weights
    best_val_loss = float('inf')  # Initialize best validation loss as infinity
    epochs_without_improvement = 0  # Counter for epochs without improvement

    # Define the learning rate scheduler (reduce LR on plateau)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    val_losses = []  # List to store validation losses for plotting

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        print(f'Starting epoch {epoch+1}/{num_epochs}')

        # Training loop
        for i, batch in enumerate(train_loader):
            if i >= steps:  # Break after the defined number of steps
                break
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation at the end of each epoch
        val_loss = validate_model(model, val_loader, criterion)
        val_losses.append(val_loss)  # Log the validation loss

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss  # Update best validation loss
            best_model_wts = copy.deepcopy(model.state_dict())  # Save the best model weights
            epochs_without_improvement = 0  # Reset counter
        else:
            epochs_without_improvement += 1
            print(f'No improvement in validation loss for {epochs_without_improvement} epoch(s).')

        if epochs_without_improvement >= patience:
            print(f'Early stopping after {epochs_without_improvement} epochs without improvement.')
            break

        # Update the learning rate based on validation loss
        scheduler.step(val_loss)

    # Load best model weights before returning
    model.load_state_dict(best_model_wts)
    print('Training complete. Best Validation Loss:', best_val_loss)
    
    return model, val_losses  # Return the model and the validation losses

def validate_model(model, val_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    return val_loss / len(val_loader)  # Return average validation loss

## Test function

Test function that generates the testing values that are necessary for the evaluation of the dataset.

In [8]:
def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    threshold = 0.5  # Threshold for multi-label classification
    
    with torch.no_grad():
        for batch in test_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) > threshold).float()

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [ ]:
torch.cuda.is_available()

## Testing for the best Optimizer

In [16]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from torchvision.models import ResNet18_Weights

def test_optimizer(opt_name, val_losses_storage):
    # Hyperparameters
    batch_size = 16
    learning_rate = 0.001
    num_epochs = 25
    steps_per_epoch = 500
    image_size = 256
    num_classes = 15

    # Paths
    train_csv = './labels/train_metadata.csv'
    val_csv = './labels/val_metadata.csv'
    image_dir = './resized_images'

    # Data Loaders
    dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

    # Loss function
    criterion = nn.BCEWithLogitsLoss()

    print(f'Testing optimizer: {opt_name}')

    # Model initialization
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    # Get the optimizer
    parameters = model.parameters()
    optimizer = get_optimizer(parameters, opt_name, learning_rate)

    # Train the model and get validation losses
    _, val_losses = train_model(model, dataloaders['train'], dataloaders['val'], criterion, optimizer, num_epochs, steps_per_epoch)

    # Store validation losses for this optimizer
    val_losses_storage[opt_name] = val_losses

    return val_losses_storage


In [52]:
import matplotlib.pyplot as plt

def plot(val_losses, message,ylabel):
    plt.figure(figsize=(10, 6))

    for name, val_losses in val_losses.items():
        plt.plot(val_losses, label=f'{name}')

    plt.title(message)
    plt.xlabel('Epochs')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.show()

In [41]:
val_losses_storage = {}

In [ ]:
# Testing SGD optimizer
val_losses_storage = test_optimizer('SGD', val_losses_storage)

In [ ]:
# Testing SGD optimizer
val_losses_storage = test_optimizer('SGD_Nesterov', val_losses_storage)

In [ ]:
# Testing SGD optimizer
val_losses_storage = test_optimizer('Adadelta', val_losses_storage)

In [ ]:
# Testing SGD optimizer
val_losses_storage = test_optimizer('Adam', val_losses_storage)

In [ ]:
# Testing SGD optimizer
val_losses_storage = test_optimizer('AdamW', val_losses_storage)

In [ ]:
# Testing SGD optimizer
val_losses_storage = test_optimizer('RMSprop', val_losses_storage)

In [ ]:
plot(val_losses_storage,'Validation Loss vs Epochs for Different Optimizers','Validation Loss')

## Testing for the best learning rate and batch size combination

In [47]:
import torch
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score
import time
import numpy as np

def train_model_logs(device, model, train_loader, val_loader, criterion, optimizer, num_epochs, steps=1000, patience=5):
    model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    # Define the learning rate scheduler (reduce LR on plateau)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    val_losses = []  # List to store validation losses for plotting
    val_auc_scores = []  # List to store AUC scores for each epoch
    time_per_epoch = []  # List to store elapsed time per epoch
    lr_per_epoch = [] # List of Lr evlution per epoch

    for epoch in range(num_epochs):
        start_time = time.time()  # Start time for epoch timing

        model.train()
        running_loss = 0.0

        print(f'Starting epoch {epoch+1}/{num_epochs}')

        # Training loop
        for i, batch in enumerate(train_loader):
            #print(i)
            if i >= steps:
                #print('break')
                break
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        elapsed_time_train = time.time() - start_time  # Calculate elapsed time
        

        # Validation at the end of each epoch
        start_time = time.time()  # Start time for epoch timing
        val_loss, val_auc = validate_model_logs(model, val_loader, criterion)

        elapsed_time_val = time.time() - start_time  # Calculate elapsed time

        val_losses.append(val_loss)  # Log the validation loss
        val_auc_scores.append(val_auc)  # Log the AUC scores
        time_per_epoch.append((elapsed_time_train,elapsed_time_val))  # Epoch elapsed time

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, AUC: {val_auc:.4f}, Time on train: {elapsed_time_train:.2f}s, Time on val: {elapsed_time_val:.2f}s')

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            print(f'No improvement in validation loss for {epochs_without_improvement} epoch(s).')

        if epochs_without_improvement >= patience:
            print(f'Early stopping after {epochs_without_improvement} epochs without improvement.')
            break

        # Update the learning rate based on validation loss
        scheduler.step(val_loss)

        new_lr = scheduler.get_last_lr()
        lr_per_epoch.append(new_lr)
        print(f'Learning rate is updated to {new_lr}')



    model.load_state_dict(best_model_wts)
    print('Training complete. Best Validation Loss:', best_val_loss)

    return model, val_losses, val_auc_scores, time_per_epoch  # Return the model and all logs

def validate_model_logs(model, val_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    val_loss = 0.0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            outputs = model(images)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_outputs.append(outputs.cpu().detach().numpy())
            all_labels.append(labels.cpu().detach().numpy())

    val_loss /= len(val_loader)  # Average validation loss
    all_outputs = np.concatenate(all_outputs)  # Concatenate outputs for AUC calculation
    all_labels = np.concatenate(all_labels)  # Concatenate labels for AUC calculation

    # Calculate AUC for each label
    auc_scores = []
    for i in range(all_labels.shape[1]):  # Assuming all_labels is shape [num_samples, num_labels]
        if np.unique(all_labels[:, i]).size > 1:  # Check for both classes in the label
            auc = roc_auc_score(all_labels[:, i], all_outputs[:, i])
            auc_scores.append(auc)
        else:
            auc_scores.append(np.nan)  # If only one class is present, AUC is undefined

    mean_auc = np.nanmean(auc_scores)  # Calculate mean AUC ignoring NaN values
    return val_loss, mean_auc  # Return average validation loss and mean AUC


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models
from torchvision.models import ResNet18_Weights

# Hyperparameters
batch_sizes = [32, 64, 128, 256, 512]
learning_rates = [1e-4, 5e-4, 1e-3, 1e-2, 1e-1]
num_epochs = 15
iter_per_epoch = 8192
image_size = 256
num_classes = 15

# Paths
train_csv = './labels/train_metadata_positive.csv'
val_csv = './labels/val_metadata_positive.csv'
image_dir = './resized_images'

# Storage matrices
final_val_losses = np.zeros((len(batch_sizes), len(learning_rates)))
final_val_aucs = np.zeros((len(batch_sizes), len(learning_rates)))
epoch_times_matrix = np.zeros((len(batch_sizes), len(learning_rates)))

# Function to map batch_size and lr indices
def get_index(batch_size, lr):
    batch_index = batch_sizes.index(batch_size)
    lr_index = learning_rates.index(lr)
    return batch_index, lr_index

In [ ]:
# Test each combination of batch size and learning rate
for batch_size in [32, 64]:
    for lr in learning_rates:
        print(f'Starting test with batch size = {batch_size} and learning rate = {lr}')

        # Data Loaders
        dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

        # Model setup
        model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

        # Loss function and optimizer
        criterion = nn.BCEWithLogitsLoss()
        optimizer = get_optimizer(model.parameters(), 'Adam', lr)

        final_epoch_iter = iter_per_epoch/batch_size

        print(f'The final epoch iterations are {final_epoch_iter}')

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Train the model and log results
        model, val_losses, val_aucs, epoch_times = train_model_logs(device, model, dataloaders['train'], dataloaders['val'], criterion, optimizer, num_epochs, final_epoch_iter)

        # Get the index for storing the results in the matrix
        batch_index, lr_index = get_index(batch_size, lr)

        # Store the final validation loss, mean AUC, and average epoch time
        final_val_losses[batch_index, lr_index] = val_losses[-1]  # Final validation loss of the last epoch
        final_val_aucs[batch_index, lr_index] = val_aucs[-1]  # Final AUC of the last epoch
        epoch_times_matrix[batch_index, lr_index] = np.mean(epoch_times)  # Mean time per epoch

In [ ]:

# Display the final matrices
print("\nFinal Validation Losses (Rows = Batch Sizes, Cols = Learning Rates):")
print(final_val_losses)

print("\nFinal Validation AUCs (Rows = Batch Sizes, Cols = Learning Rates):")
print(final_val_aucs)

print("\nEpoch Times (Rows = Batch Sizes, Cols = Learning Rates):")
print(epoch_times_matrix)

In [ ]:
# Test each combination of batch size and learning rate
for batch_size in [128, 256]:
    for lr in learning_rates:
        print(f'Starting test with batch size = {batch_size} and learning rate = {lr}')

        # Data Loaders
        dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

        # Model setup
        model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

        # Loss function and optimizer
        criterion = nn.BCEWithLogitsLoss()
        optimizer = get_optimizer(model.parameters(), 'Adam', lr)

        final_epoch_iter = iter_per_epoch/batch_size

        print(f'The final epoch iterations are {final_epoch_iter}')

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Train the model and log results
        model, val_losses, val_aucs, epoch_times = train_model_logs(device, model, dataloaders['train'], dataloaders['val'], criterion, optimizer, num_epochs, final_epoch_iter)

        # Get the index for storing the results in the matrix
        batch_index, lr_index = get_index(batch_size, lr)

        # Store the final validation loss, mean AUC, and average epoch time
        final_val_losses[batch_index, lr_index] = val_losses[-1]  # Final validation loss of the last epoch
        final_val_aucs[batch_index, lr_index] = val_aucs[-1]  # Final AUC of the last epoch
        epoch_times_matrix[batch_index, lr_index] = np.mean(epoch_times)  # Mean time per epoch

In [ ]:
# Test each combination of batch size and learning rate
for batch_size in [512]:
    for lr in learning_rates:
        print(f'Starting test with batch size = {batch_size} and learning rate = {lr}')

        # Data Loaders
        dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

        # Model setup
        model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

        # Loss function and optimizer
        criterion = nn.BCEWithLogitsLoss()
        optimizer = get_optimizer(model.parameters(), 'Adam', lr)

        final_epoch_iter = iter_per_epoch/batch_size

        print(f'The final epoch iterations are {final_epoch_iter}')

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Train the model and log results
        model, val_losses, val_aucs, epoch_times = train_model_logs(device, model, dataloaders['train'], dataloaders['val'], criterion, optimizer, num_epochs, final_epoch_iter)

        # Get the index for storing the results in the matrix
        batch_index, lr_index = get_index(batch_size, lr)

        # Store the final validation loss, mean AUC, and average epoch time
        final_val_losses[batch_index, lr_index] = val_losses[-1]  # Final validation loss of the last epoch
        final_val_aucs[batch_index, lr_index] = val_aucs[-1]  # Final AUC of the last epoch
        epoch_times_matrix[batch_index, lr_index] = np.mean(epoch_times)  # Mean time per epoch

In [ ]:

# Display the final matrices
print("\nFinal Validation Losses (Rows = Batch Sizes, Cols = Learning Rates):")
print(final_val_losses)

print("\nFinal Validation AUCs (Rows = Batch Sizes, Cols = Learning Rates):")
print(final_val_aucs)

print("\nEpoch Times (Rows = Batch Sizes, Cols = Learning Rates):")
print(epoch_times_matrix)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models
from torchvision.models import ResNet18_Weights

# Hyperparameters
batch_size = 128
learning_rate = 5e-4
num_epochs = 10
iter_per_epoch = 500
image_sizes = [256,224,128,64]
num_classes = 15

# Paths
train_csv = './labels/train_metadata.csv'
val_csv = './labels/val_metadata.csv'
image_dir = './resized_images'

# Storage matrices
final_val_losses = []
final_val_aucs = []
final_epoch_times = []

# Function to map batch_size and lr indices
def get_index(batch_size, lr):
    batch_index = batch_sizes.index(batch_size)
    lr_index = learning_rates.index(lr)
    return batch_index, lr_index

# Test each combination of batch size and learning rate
for image_size in image_sizes:
    print(f'Starting test with image size = {image_size}')

    # Data Loaders
    dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

    # Model setup
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    # Loss function and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = get_optimizer(model.parameters(), 'Adam', learning_rate)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Train the model and log results
    model, val_losses, val_aucs, epoch_times = train_model_logs(device, model, dataloaders['train'], dataloaders['val'], criterion, optimizer, num_epochs, iter_per_epoch)

    final_val_losses.append(val_losses)
    final_val_aucs.append(val_aucs)
    final_epoch_times.append(epoch_times)

In [22]:
size_exp = {}
for i in range(len(image_sizes)):
    size_exp[image_sizes[i]] = final_val_losses[i]

In [ ]:
plot(size_exp, 'Validation Loss vs Epochs for different Image sizes','Validation Loss')

In [12]:
import torch
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import time
import numpy as np
import pandas as pd

def train_model_history(device, model, train_loader, val_loader, criterion, optimizer, num_epochs, steps=1000, patience=5):
    model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    # Define the learning rate scheduler (reduce LR on plateau)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    # DataFrame to store history
    history = pd.DataFrame(columns=['epoch', 'val_loss', 'val_auc', 'precision', 'recall', 'f1_score', 'lr', 'train_time', 'val_time', 'epoch_time'])

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        print(f'Starting epoch {epoch+1}/{num_epochs}')
        
        start_time = time.time()  # Start time for training phase

        # Training loop
        for i, batch in enumerate(train_loader):
            if i >= steps:
                break
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_time = time.time() - start_time  # End time for training phase

        # Validation at the end of each epoch
        start_time_val = time.time()  # Start time for validation phase

        val_loss, val_auc, val_precision, val_recall, val_f1 = validate_model_history(model, val_loader, criterion)

        val_time = time.time() - start_time_val  # End time for validation phase

        epoch_time = time.time() - start_time  # End time for the entire epoch

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, AUC: {val_auc:.4f}, '
              f'Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-score: {val_f1:.4f}, '
              f'Training Time: {train_time:.2f}s, Validation Time: {val_time:.2f}s, Total Time: {epoch_time:.2f}s')

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            print(f'No improvement in validation loss for {epochs_without_improvement} epoch(s).')

        if epochs_without_improvement >= patience:
            print(f'Early stopping after {epochs_without_improvement} epochs without improvement.')
            break

        # Update the learning rate based on validation loss
        scheduler.step(val_loss)
        current_lr = scheduler.optimizer.param_groups[0]['lr']

        # Log metrics into the DataFrame, including time information
        history = pd.concat([history, pd.DataFrame({
            'epoch': [epoch + 1],
            'val_loss': [val_loss],
            'val_auc': [val_auc],
            'precision': [val_precision],
            'recall': [val_recall],
            'f1_score': [val_f1],
            'lr': [current_lr],
            'train_time': [train_time],
            'val_time': [val_time],
            'epoch_time': [epoch_time]
        })], ignore_index=True)

        print(f'Learning rate is updated to {current_lr}')

    model.load_state_dict(best_model_wts)
    print('Training complete. Best Validation Loss:', best_val_loss)

    return model, history  # Return the model and the history DataFrame

def validate_model_history(model, val_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    val_loss = 0.0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            outputs = model(images)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_outputs.append(torch.sigmoid(outputs).cpu().detach().numpy())  # Apply sigmoid
            all_labels.append(labels.cpu().detach().numpy())

    val_loss /= len(val_loader)  # Average validation loss
    all_outputs = np.concatenate(all_outputs)  # Concatenate outputs for AUC calculation
    all_labels = np.concatenate(all_labels)  # Concatenate labels for AUC calculation

    # Threshold outputs for binary predictions
    all_preds = (all_outputs > 0.5).astype(int)

    # Calculate AUC for each label
    auc_scores = []
    for i in range(all_labels.shape[1]):  # Assuming all_labels is shape [num_samples, num_labels]
        if np.unique(all_labels[:, i]).size > 1:  # Check for both classes in the label
            auc = roc_auc_score(all_labels[:, i], all_outputs[:, i])
            auc_scores.append(auc)
        else:
            auc_scores.append(np.nan)  # If only one class is present, AUC is undefined

    mean_auc = np.nanmean(auc_scores)  # Calculate mean AUC ignoring NaN values

    # Calculate precision, recall, and F1-score
    precision = precision_score(all_labels, all_preds, average='micro')
    recall = recall_score(all_labels, all_preds, average='micro')
    f1 = f1_score(all_labels, all_preds, average='micro')

    return val_loss, mean_auc, precision, recall, f1  # Return all metrics


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models
from torchvision.models import ResNet18_Weights

# Hyperparameters
batch_size = 128
learning_rate = 5e-4
num_epochs = 5
iter_per_epoch = 50
image_size = 256
num_classes = 15

# Paths
train_csv = './labels/train_metadata.csv'
val_csv = './labels/val_metadata.csv'
image_dir = './resized_images'

# Data Loaders
dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

print(class_counts)
print(effective_weights(class_counts))
print(torch.sum(effective_weights(class_counts)))

# Model setup
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loss function and optimizer
counts = np.array(class_counts)
loss_fn = get_loss(loss_type='asymmetric', counts=counts, device=device)
print(loss_fn)
optimizer = get_optimizer(model.parameters(), 'Adam', learning_rate)


# Train the model and log results
model, history = train_model_history(device, model, dataloaders['train'], dataloaders['val'], loss_fn, optimizer, num_epochs, iter_per_epoch)

In [ ]:
import torch

# Convert pred and labels to PyTorch tensors
pred_tensor = torch.tensor([[-7.3645e-01,  5.7881e-01, -2.2020e-01, -5.1094e-01,  2.9868e-01,
                             2.6686e-01,  3.5119e-02, -6.2442e-01, -2.2921e-01,  2.8613e-01,
                             3.6774e-01, -4.4509e-01, -1.0131e+00,  2.7856e-02,  1.0464e+00],
                            [ 5.1233e-01, -2.9292e-02, -8.7824e-02, -8.9985e-01,  6.1635e-01,
                             1.0113e+00,  1.2067e-02, -1.3016e+00,  2.0202e-01, -3.5548e-01,
                            -5.4201e-01, -5.7460e-01, -4.4881e-01,  9.9741e-01,  1.1251e+00],
                            [ 5.9710e-01,  6.8125e-02, -2.9039e-01, -7.9447e-01,  5.1311e-01,
                            -3.1247e-01, -1.2926e-01, -1.1064e+00, -6.4016e-01, -9.4150e-01,
                             1.9711e-01, -4.3601e-01, -6.0893e-01,  6.6210e-02,  5.9627e-01],
                            [-1.4584e-01,  1.2339e-01, -9.8124e-02, -5.5506e-01,  6.5351e-01,
                             3.3160e-01, -1.7156e-01, -5.0052e-01, -3.9164e-01,  2.6694e-01,
                             5.2570e-01,  2.9331e-02, -7.3786e-01,  1.6505e-01,  8.5180e-01],
                            [ 3.7813e-01,  1.6255e-01, -1.9692e-01, -5.3570e-01,  8.5489e-01,
                             1.0920e+00,  3.0120e-01, -9.9126e-01, -6.8351e-02, -5.0176e-01,
                            -5.2612e-01, -7.2143e-01, -6.2451e-01,  5.1360e-01,  1.0608e+00],
                            [ 7.1772e-01,  5.5271e-01, -3.9907e-01, -1.2032e+00,  5.2024e-01,
                             2.2213e-01, -7.2604e-02, -1.0538e+00, -3.7901e-01, -5.6711e-01,
                             2.4622e-01, -3.0617e-01, -2.3560e-02,  1.0056e-01,  7.4420e-01]])

labels_tensor = torch.tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                              [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                              [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
                              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
                              [0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
                              [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# Paths
train_csv = './labels/train_metadata.csv'
val_csv = './labels/val_metadata.csv'
image_dir = './resized_images'

print(f'Starting test with bce')

# Data Loaders
dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

print(class_counts)
print(effective_weights(class_counts))
print(torch.sum(effective_weights(class_counts)))

# Loss function and optimizer
counts = np.array(class_counts)
for loss in ['asymmetric','focal','bce_w','bce']:
    loss_fn = get_loss(loss_type=loss, counts=counts)
    print(loss_fn)

    curr_loss = loss_fn(pred_tensor, labels_tensor)
    print(f"{loss} Loss: {curr_loss.item()}")

In [13]:
# Function for testing different loss types
def get_loss(loss_type='bce', counts=None, beta=0.99, device='cpu'):
    
    if loss_type == 'bce':
        loss_fn = nn.BCEWithLogitsLoss()
    elif loss_type == 'bce_w':
        if counts is not None:
            class_weights = effective_weights(counts, beta=beta).to(device)
            loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)
        else:
            print("You didn't provide weights so we proceed with normal BCE")
            loss_fn = nn.BCEWithLogitsLoss()
    elif loss_type == 'asymmetric':
        loss_fn = AsymmetricLoss()
    elif loss_type == 'asymmetric_avg':
        loss_fn = AsymmetricLoss(average=True)
    elif loss_type == 'asl1':
        loss_fn = AsymmetricLoss(gamma_neg=2, gamma_pos=1, clip=0)
    elif loss_type == 'asl2':
        loss_fn = AsymmetricLoss(gamma_neg=4, gamma_pos=1, clip=0.05)
    elif loss_type == 'asl3':
        loss_fn = AsymmetricLoss(gamma_neg=8, gamma_pos=1, clip=0.05)
    elif loss_type == 'asl4':
        loss_fn = AsymmetricLoss(gamma_neg=2, gamma_pos=1, clip=0.025)
    elif loss_type == 'asl5':
        loss_fn = AsymmetricLoss(gamma_neg=4, gamma_pos=1, clip=0)
    elif loss_type == 'focal':
        loss_fn = AsymmetricLoss(gamma_neg=2, gamma_pos=2, clip=0)
    
    else:
        raise ValueError(f"Unknown loss_type '{loss_type}'. Use 'bce' or 'asymmetric'.")
    
    return loss_fn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models
from torchvision.models import ResNet18_Weights

# Hyperparameters
batch_size = 128
learning_rate = 5e-4
num_epochs = 25
iter_per_epoch = 64
image_size = 256
num_classes = 15

# Paths
train_csv = './labels/train_metadata.csv'
val_csv = './labels/val_metadata.csv'
image_dir = './resized_images'

# Storage matrices
histories = {}


# Test each combination of batch size and learning rate
for loss_func in ['bce','asymmetric','asl1','asl2','asl4','asl5']: # 
    print(f'Starting test with loss = {loss_func}')

    # Data Loaders
    dataloaders, dataset_sizes, class_counts = make_data_loaders(train_csv, val_csv, image_dir, batch_size, image_size)

    # Model setup
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Loss function and optimizer
    counts = np.array(class_counts)
    loss_fn = get_loss(loss_type=loss_func, counts=counts, device=device)
    optimizer = get_optimizer(model.parameters(), 'Adam', learning_rate)

    # Train the model and log results
    model, history = train_model_history(device, model, dataloaders['train'], dataloaders['val'], loss_fn, optimizer, num_epochs, iter_per_epoch)

    histories[loss_func] = history

In [ ]:
for key, df in histories.items():
    # Save each DataFrame as "key.csv"
    filename = f"./tests/loss_func/{key}.csv"
    df.to_csv(filename, index=False)  # Set index=False if you don't want to store the index in the CSV
    print(f"Saved {key} as {filename}")

In [ ]:
histories = {}
for key in ['bce','bce_w','focal','asymmetric','asymmetric_avg','asl1','asl2','asl3']:
    # Save each DataFrame as "key.csv"
    filename = f"./tests/loss_func/{key}.csv"
    histories[key] = pd.read_csv(filename)  
    print(f"Retrieved {key} from {filename}")

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(history_dict):
    # Create 4 subplots for AUC, Precision, Recall, and F1 score
    fig, axs = plt.subplots(4, 1, figsize=(12, 20))
    
    # Plot for each key in the dictionary
    for key, history in history_dict.items():
        epochs = history['epoch']
        
        # Plot AUC
        axs[0].plot(epochs, history['val_auc'], label=key)
        axs[0].set_title('AUC per Epoch')
        axs[0].set_xlabel('Epoch')
        axs[0].set_ylabel('AUC')
        
        # Plot Precision
        axs[2].plot(epochs, history['precision'], label=key)
        axs[2].set_title('Precision per Epoch')
        axs[2].set_xlabel('Epoch')
        axs[2].set_ylabel('Precision')
        
        # Plot Recall
        axs[1].plot(epochs, history['recall'], label=key)
        axs[1].set_title('Recall per Epoch')
        axs[1].set_xlabel('Epoch')
        axs[1].set_ylabel('Recall')
        
        # Plot F1 Score
        axs[3].plot(epochs, history['f1_score'], label=key)
        axs[3].set_title('F1 Score per Epoch')
        axs[3].set_xlabel('Epoch')
        axs[3].set_ylabel('F1 Score')
    
    # Add legend to all plots
    for ax in axs.flat:
        ax.legend()
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

plot_metrics(histories)
